In [7]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import time
from tqdm import tqdm
from tabulate import tabulate

from sklearn.metrics.pairwise import polynomial_kernel
from sklearn.metrics.pairwise import euclidean_distances

In [8]:
import pandas as pd

df = pd.read_csv('zipcombo.csv') #(9298, 258)
df.head() 

,label,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,256
0,6.0,-1.0,-1.0,-1.0,-1.000,-1.000,-1.000,-1.000,-0.631,0.862,...,0.304,0.823,1.000,0.482,-0.474,-0.991,-1.000,-1.000,-1.000,-1.0
1,5.0,-1.0,-1.0,-1.0,-0.813,-0.671,-0.809,-0.887,-0.671,-0.853,...,-0.671,-0.671,-0.033,0.761,0.762,0.126,-0.095,-0.671,-0.828,-1.0
2,4.0,-1.0,-1.0,-1.0,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,...,-1.000,-1.000,-1.000,-0.109,1.000,-0.179,-1.000,-1.000,-1.000,-1.0
3,7.0,-1.0,-1.0,-1.0,-1.000,-1.000,-0.273,0.684,0.960,0.450,...,-0.318,1.000,0.536,-0.987,-1.000,-1.000,-1.000,-1.000,-1.000,-1.0
4,3.0,-1.0,-1.0,-1.0,-1.000,-1.000,-0.928,-0.204,0.751,0.466,...,0.466,0.639,1.000,1.000,0.791,0.439,-0.199,-0.883,-1.000,-1.0


In [9]:
X = df[df.columns[1:]].values.copy()
y = df['label'].values.copy()

In [10]:
#function to perform mapping with the polynomial kernel
def polynomial(X_i, X_j, d):
    '''
    d: dimension fo the polynomial kernel
    '''
    K = ( X_i.dot(X_j.T) ) ** d
    return K

#a fast implementaion of the polynomial kernel
def fast_polynomial(X_i, X_j, d):
    '''
    d: dimension fo the polynomial kernel
    '''
    K = polynomial_kernel(X_i, Y=X_j, degree=d)
    return K 

#function to perform mapping with the gaussian kernel
def gaussian(X_i, X_j, c):
    '''
    c: width of the gaussian kernel
    '''
    m, n = len(X_i), len(X_j)
    K = np.zeros((m,n))            #initialize the mapped feature matrix
    for i in range(m):
        for j in range(n):
            K[i][j] = np.exp( -c * np.linalg.norm(X_i[i]-X_j[j]) ** 2 )
    return K

#a faster implementation of the gaussian kernel
def fast_gaussian(X_i, X_j, c):
    '''
    c: width of the gaussian kernel
    '''
    K = euclidean_distances(X_i, X_j)
    K = np.exp( -c * K ** 2 )
    return K

In [11]:
def shuffle_data(X, y, seed=None):
    if seed:                         #set a random seed for reproducable results
        np.random.seed(seed)
    idx = np.arange(len(X))          
    np.random.shuffle(idx)           #shuffle the index
    return X[idx], y[idx]

#function to perform train-test split
def train_test_split(X, y, train_split, shuffle, seed=None):
    '''
    train_split: percentage of data for training
    '''
    if shuffle:                                     #shuffle the data if needed
        X, y = shuffle_data(X, y, seed)
    n_train = int(train_split*len(X))               #find the split location
    X_train, X_test = X[:n_train], X[n_train:]      
    y_train, y_test = y[:n_train], y[n_train:]
    return X_train, X_test, y_train, y_test

#function to split data for k-fold cross-validation
def KFold(X, y, k):
    m = len(X)
    n_split = int(m/k)         
    X_train_fold, X_valid_fold, y_train_fold, y_valid_fold = [], [], [], []
    for fold in range(1, k+1):
        X_valid = X[(fold-1)*n_split:fold*n_split]
        y_valid = y[(fold-1)*n_split:fold*n_split]
        X_train = np.append(X[0:(fold-1)*n_split], X[fold*n_split:], axis=0)
        y_train = np.append(y[0:(fold-1)*n_split], y[fold*n_split:], axis=0)
        X_train_fold.append(X_train)
        X_valid_fold.append(X_valid)
        y_train_fold.append(y_train)
        y_valid_fold.append(y_valid)
    return X_train_fold, X_valid_fold, y_train_fold, y_valid_fold

In [12]:
class MultiKernelPerceptron(object):
    
    def __init__(self, X_train, y_train, X_test, y_test, kernel, kernel_param, epochs):
        
        #initialize the datasets
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        
        #initialize the sizes
        self.epochs = epochs
        self.batch_size = len(self.y_train)  
        self.test_size = len(self.y_test)
        self.classes = np.unique(np.append(self.y_train, self.y_test))
        self.n_c = len(self.classes)
        
        #initialize the kernel method
        self.kernel = kernel
        self.kernel_param = kernel_param
        
        #initialize the mapped data with kernelization
        if kernel == 'p':
            self.K = fast_polynomial(self.X_train, self.X_train, self.kernel_param)
            self.K_test = fast_polynomial(self.X_test, self.X_train, self.kernel_param)
        if kernel =='g':
            self.K = fast_gaussian(self.X_train, self.X_train, self.kernel_param)
            self.K_test = fast_gaussian(self.X_test, self.X_train, self.kernel_param)   
        
        #initialize the alpha matrix: n_c × m
        self.alpha = np.zeros(shape=(self.n_c, self.batch_size))
        
        #intialize the confidence matrices: m × n_c
        self.confidence = np.zeros(shape=(self.batch_size, self.n_c))
        self.confidence_test = np.zeros(shape=(self.batch_size, self.n_c))
        
        #initialize the confusion matrix: n_c × n_c
        self.confusion = np.zeros(shape=(self.n_c, self.n_c))
        

    def predict(self, i, data):
        if data == 'train':
            confidence = np.dot(self.alpha, self.K[i])             #compute the confidence
            self.confidence[i,:] = confidence                      #store the results in the condifence matrix
            y_pred = np.argmax(confidence)                         #make the final prediction
        elif data == 'test':
            confidence = np.dot(self.alpha, self.K_test[i])
            self.confidence_test[i,:] = confidence
            y_pred = np.argmax(confidence)
        return confidence, y_pred
        
    
    def train(self):
        for _ in range(self.epochs):                    #train the algorithm over 10 epoches
            errors = 0                                  #reset the count of mistakes to 0 at the beginning of each epoch
            for i in range(self.batch_size):            #sequentially train the algorithm over each sample
                confidence, y_pred = self.predict(i, 'train')      #make prediction
                label = int(self.y_train[i])
                if int(y_pred) != label:                           #if the prediction does not match the label
                    errors += 1                                    #increase the count of mistakes by 1
                    self.alpha[label, i] += 1                      #update the alpha 
                    self.alpha[y_pred, i] -= 1
            error_rate = (errors/self.batch_size) * 100            #compute the error rate for this epoch
        return error_rate                                          #return the training error rate on the last epoch
    
    
    def test(self):
        errors = 0
        for i in range(self.test_size):
            confidence, y_pred = self.predict(i, 'test') 
            label = int(self.y_test[i])
            if int(y_pred) != label:
                errors += 1
                self.confusion[label, int(y_pred)] += 1       #update the copnfusion matrix 
        for i in range(self.n_c):                             #for each class
            count = list(self.y_test).count(i)                      #count the number of class i in the test set
            self.confusion[i] = self.confusion[i] / count     #compute the error rates
        error_rate = (errors/self.test_size) * 100 
        return error_rate 

In [13]:
#initialization
np.random.seed(0)
train_split = 0.8
d_list = np.arange(1,8)
train_error, train_std, test_error, test_std = [], [], [], []

for d in d_list:               #for different dimensions of the polynomial kernel
    single_run_train_errors, single_run_test_errors = [], []
    for run in tqdm(range(20)):      #perform 20 runs
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_split, shuffle=True)     #randomly split the data
        clf = MultiKernelPerceptron(X_train, y_train, X_test, y_test, kernel='p', kernel_param=d, epochs=5)
        train_e = clf.train()                            #compute the training error on the 80% data
        single_run_train_errors.append(train_e)
        test_e = clf.test()                              #compute the test error on the 20% data
        single_run_test_errors.append(test_e) 
    train_error.append(np.mean(single_run_train_errors))
    train_std.append(np.std(single_run_train_errors))
    test_error.append(np.mean(single_run_test_errors))
    test_std.append(np.std(single_run_test_errors))
    print('Polynomial order: ', d, ', mean train error: ', train_error[-1], ', mean test error: ', test_error[-1])

100%|██████████| 20/20 [00:13<00:00,  1.46it/s]


Polynomial order:  1 , mean train error:  9.023931164291476 , mean test error:  9.631720430107526


100%|██████████| 20/20 [00:13<00:00,  1.51it/s]


Polynomial order:  2 , mean train error:  4.768082817961817 , mean test error:  6.908602150537634


100%|██████████| 20/20 [00:21<00:00,  1.06s/it]


...

Polynomial order:  5 , mean train error:  1.1118580263511697 , mean test error:  4.048387096774194


100%|██████████| 20/20 [00:21<00:00,  1.05s/it]


Polynomial order:  6 , mean train error:  0.8120462489916644 , mean test error:  3.7876344086021505


100%|██████████| 20/20 [00:21<00:00,  1.05s/it]

Polynomial order:  7 , mean train error:  0.6460069911266471 , mean test error:  3.561827956989247


In [14]:
#tabulate the results
data = []
for i in range(len(d_list)):
    train_result = f"{'{:.4f}'.format(train_error[i])}±{'{:.4f}'.format(train_std[i])}"
    test_result = f"{'{:.4f}'.format(test_error[i])}±{'{:.4f}'.format(test_std[i])}"
    result = [int(d_list[i]), train_result, test_result]
    data.append(result)
print(tabulate(data, 
               headers = ["d", "Mean Train Error Rates (%)", "Mean Test Error Rates (%)"], 
               tablefmt = "simple_outline",
               stralign = "center"))

┌─────┬──────────────────────────────┬─────────────────────────────┐
│   d │  Mean Train Error Rates (%)  │  Mean Test Error Rates (%)  │
├─────┼──────────────────────────────┼─────────────────────────────┤
│   1 │        9.0239±0.2944         │        9.6317±1.8428        │
│   2 │        4.7681±0.2877         │        6.9086±1.5810        │
│   3 │        2.5746±0.1838         │        4.9274±0.6437        │
│   4 │        1.6510±0.1692         │        4.7527±0.7367        │
│   5 │        1.1119±0.1261         │        4.0484±0.6295        │
│   6 │        0.8120±0.1401         │        3.7876±0.6084        │
│   7 │        0.6460±0.1170         │        3.5618±0.4026        │
└─────┴──────────────────────────────┴─────────────────────────────┘


In [15]:
#initialization 
np.random.seed(1)
train_split = 0.8
d_stars, test_errors = [], []

for run in tqdm(range(20)):          #perform 20 runs
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_split, shuffle=True)     #randomly split the data
    d_star, best_error = 0, float('inf')            #initialize the best d and best error
    for d in d_list:           #try different dimensions of the polynomial kernel
        valid_errors = []      #initialize a list to store validation errors
        X_train_frold, X_valid_fold, y_train_fold, y_valid_fold = KFold(X_train, y_train, 5)    #split the training set into 5 folds
        for Xtrain, Xvalid, ytrain, yvalid in zip(X_train_frold, X_valid_fold, y_train_fold, y_valid_fold):
            clf = MultiKernelPerceptron(Xtrain, ytrain, Xvalid, yvalid, kernel='p', kernel_param=d, epochs=5)
            train_error = clf.train()               #train the classifier
            valid_errors.append(clf.test())         #evalaute the classifier on the validation set
        if np.mean(valid_errors) < best_error:      #if the current d value gives a lower error rate
            best_error = np.mean(valid_errors)      #update the best error so far
            d_star = d                              #update the best d value so far
    #retrain the classifier on the entire training set with the best d value
    clf = MultiKernelPerceptron(X_train, y_train, X_test, y_test, kernel='p', kernel_param=d_star, epochs=5)   
    train_error = clf.train()
    test_error = clf.test()
    test_errors.append(test_error)
    d_stars.append(d_star)
    print('d*: ', d_star, ', test error rate: ', test_errors[-1])

  5%|▌         | 1/20 [00:33<10:35, 33.42s/it]

d*:  7 , test error rate:  3.6021505376344085


 10%|█         | 2/20 [01:07<10:06, 33.71s/it]

d*:  7 , test error rate:  2.849462365591398


 15%|█▌        | 3/20 [01:41<09:38, 34.01s/it]

...

d*:  7 , test error rate:  2.5806451612903225


 95%|█████████▌| 19/20 [10:41<00:33, 33.62s/it]

d*:  7 , test error rate:  3.3333333333333335


100%|██████████| 20/20 [11:14<00:00, 33.71s/it]

d*:  7 , test error rate:  2.6344086021505375
